In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
n_rounds= 11

In [ ]:
file_path = f'./Data/2class_HIGHT_round{n_rounds}_new.csv'

In [ ]:
dataset= pd.read_csv(file_path, header= None)

In [ ]:
dataset

In [ ]:
X= dataset.iloc[:, :-1]
y= dataset.iloc[:, -1]

In [ ]:
X=X.values
y=y.values

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.50, random_state = 20)
X_val, X_test, y_val, y_test= train_test_split(X_test, y_test, test_size = 0.40, random_state = 20)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

In [ ]:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_val, return_counts=True))
print(np.unique(y_test, return_counts=True))

In [ ]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, dim_e, n_attn, d_units, rate=0.1):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=n_attn, key_dim=dim_e)
        self.denselayer = tf.keras.Sequential(
            [tf.keras.layers.Dense(d_units, activation="relu"), 
             tf.keras.layers.Dense(dim_e)]
        )
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout_1 = tf.keras.layers.Dropout(rate)
        self.dropout_2 = tf.keras.layers.Dropout(rate)

    def call(self, sequence):
        attention = self.attention(sequence, sequence)
        attention = self.dropout_1(attention)
        ln = self.norm_1(sequence + attention)
        dense_trans = self.denselayer(ln)
        dense_trans = self.dropout_2(dense_trans)
        return self.norm_2(ln + dense_trans)


In [ ]:
vocabulary = 256
seq_length = 24

In [ ]:
class EmbeddingBlock(tf.keras.layers.Layer):
    def __init__(self, seq_length, vocabulary, dim_e):
        super().__init__()
        self.e_token = tf.keras.layers.Embedding(input_dim=vocabulary, output_dim=dim_e) #token embedding
        self.e_pos = tf.keras.layers.Embedding(input_dim=seq_length, output_dim=dim_e) #position embedding

    def call(self, t):
        size = np.shape(t)[-1]
        index = np.arange(start=0, stop=size, step=1)
        index = self.e_pos(index)
        t = self.e_token(t)
        return t + index
    

In [ ]:

dim_e = 150  # Embedding size for each token
n_attn =  15  # Number of attention heads
d_units= 150     # Hidden layer size in feed forward network inside transformer

seq_input = tf.keras.layers.Input(shape=(seq_length,))
embedding_block = EmbeddingBlock(seq_length, vocabulary, dim_e)
trans_block = TransformerBlock(dim_e, n_attn, d_units)

l = embedding_block(seq_input)
l = trans_block(l)
l = tf.keras.layers.GlobalAveragePooling1D()(l)
l = tf.keras.layers.Dropout(0.1)(l)
l = tf.keras.layers.Dense(200, activation="relu")(l)
# l = tf.keras.layers.Dense(160, activation="relu")(l)
# l = tf.keras.layers.Dense(8, activation="relu")(l)
# l = tf.keras.layers.Dropout(0.1)(l)
pred_class = tf.keras.layers.Dense(1, activation="sigmoid")(l)

model = tf.keras.Model(inputs=seq_input, outputs=pred_class)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=150,callbacks=[EarlyStopping(monitor="val_accuracy", min_delta= 0.0003, patience= 5, verbose= 0, baseline= None, restore_best_weights= True, start_from_epoch=0)], epochs=600, validation_data=(X_val, y_val))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
preds_test = model.predict(X_test)
preds_train= model.predict(X_train)

In [ ]:
for i in range(len(preds_test)):
    preds_test[i] = 1 if (preds_test[i]>=0.5) else 0

for i in range(len(preds_train)):
    preds_train[i] = 1 if (preds_train[i]>=0.5) else 0   

In [ ]:
preds_train

In [ ]:
preds_test

In [ ]:
print(f"{accuracy_score(y_train, preds_train) = }")
print(f"{accuracy_score(y_test, preds_test) = }")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
from sklearn import metrics
matrix = metrics.confusion_matrix(y_test[:len(preds_test)], preds_test)
print(matrix)

In [ ]:
TP=matrix[0][0]
FN=matrix[0][1]
FP=matrix[1][0]
TN=matrix[1][1]
TPR=TP/(TP+FN)
print(f'TPR={TPR}')
TNR=TN/(TN+FP)
print(f'TNR={TNR}')